In [1]:
from cnn_globals import *

from cnn_data import cnn_data, cnn_data_split

%matplotlib inline

[INFO    ] Using cmssw CMSSW_10_1_7
[INFO    ] Using numpy 1.14.1
/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-omkpbe5/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
[INFO    ] Using tensorflow 1.5.0
[INFO    ] Using keras 2.1.2-tf
[INFO    ] .. list devices: [_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456)]
[INFO    ] Using scipy 1.0.0
[INFO    ] Using sklearn 0.19.2


In [2]:
# Import images and labels
data = cnn_data_split(infile_images, test_size=0.4, shuffle=False, nentries=40000)

(images_px_train, images_px_test, images_ch_train, images_ch_test, labels_train, labels_test, parameters_train, parameters_test) = data

[INFO    ] Loading cnn data from ../test7/histos_tbe.17a.npz ...
[INFO    ] Loaded the images with shape (3535956, 50, 2),(3535956, 50, 3)
[INFO    ] Loaded the labels with shape (3535956, 3)
[INFO    ] Loaded the parameters with shape (3535956, 4)
[INFO    ] Loaded # of training and testing events: (24000, 16000)


In [3]:
use_jupyter = True

use_keras = True

if use_keras:
  from cnn_keras import define_reiam_flags, run_reiam
else:
  from cnn_estimator import define_reiam_flags, run_reiam

In [ ]:
#from absl import app
#from absl import flags
flags = tf.flags
app = tf.app

def main(_):
  data = (images_px_train, images_px_test, images_ch_train, images_ch_test, labels_train, labels_test, parameters_train, parameters_test)
  run_reiam(flags.FLAGS, data)

def run(main, argv=['lol']):
  if use_jupyter:
    argv = flags.FLAGS(argv)
    main(argv)
  else:
    app.run(main=main, argv=argv)

#tf.logging.set_verbosity(tf.logging.INFO)
define_reiam_flags()
run(main)
logger.info('DONE')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 7, 158, 48)        30144     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 158, 64)        76864     
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 158, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 158, 32)        18464     
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 158, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 52, 32)         0      

Using TensorFlow backend.
[INFO    ] Begin training ...
